In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [32]:
trainset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(dataset=trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=testset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [47]:
sample = next(iter(train_loader))
sample[0].shape, sample[1].shape

(torch.Size([32, 3, 32, 32]), torch.Size([32]))

In [33]:
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)

In [34]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [35]:
num_featues = model.fc.in_features
print(num_featues)
model.fc = nn.Linear(num_featues, 10)
model = model.to(device)

512


In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

In [37]:
for epoch in range(20):
    running_loss = 0.0
    for data in train_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    cost = running_loss / len(train_loader)
    print(f'[{epoch+1}] loss: {cost:.3f}')

torch.save(model.state_dict(), 'models/cifar10_resnet18.pth')

print('Finished Training')

[1] loss: 1.148
[2] loss: 0.808
[3] loss: 0.716
[4] loss: 0.666
[5] loss: 0.631
[6] loss: 0.603
[7] loss: 0.578
[8] loss: 0.567
[9] loss: 0.554
[10] loss: 0.546
[11] loss: 0.537
[12] loss: 0.530
[13] loss: 0.529
[14] loss: 0.523
[15] loss: 0.507
[16] loss: 0.501
[17] loss: 0.488
[18] loss: 0.488
[19] loss: 0.482
[20] loss: 0.467
Finished Training


In [40]:
model = torchvision.models.resnet18(weights=None)
num_features = model.fc.in_features
print(f'num_features: {num_features}')
model.fc = nn.Linear(in_features=num_featues, out_features=10)
model.to(device)
model.load_state_dict(torch.load('models/cifar10_resnet18.pth'))

num_features: 512


<All keys matched successfully>

In [50]:
correct = 0
total = 0
with torch.inference_mode():
    model.eval()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        total += len(labels)
        correct += (labels == outputs.argmax(axis=1)).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}')

Accuracy of the network on the 10000 test images: 86.88
